In [ ]:
%matplotlib widget
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qm import QuantumMachinesManager
from qualang_tools.results import progress_counter, fetching_tool
from qualang_tools.loops import from_array
from progress import addjob, ProgressPlot
import threading
import config_qubit as config
import importlib

In [ ]:
# Connect to the cluster (run only once)
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name)

# Connexion à la Quantum Machine

In [ ]:
# Get the QM reference (rerun every time the config is changed)
importlib.reload(config)
qm_list =  qmm.list_open_qms()
qm = qmm.get_qm(qm_list[0])
print(f"Connected to {qm.id}")

# Analyse des histogrammes de mesure

In [ ]:
# Parameters Definition
n_avg = 20000  # The number of averages
thermalization_time = 200 #in µs
waiting_time = 1 * u.us
###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  # QUA variable for the averaging loop
    I0 = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q0 = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I0_st = declare_stream()  # Stream for the 'I' quadrature
    Q0_st = declare_stream()  # Stream for the 'Q' quadrature
    I1 = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q1 = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I1_st = declare_stream()  # Stream for the 'I' quadrature
    Q1_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_avg, n + 1):  # QUA for_ loop for averaging
            # Measure the state of the resonator
            # The integration weights have changed to maximize the SNR after having calibrated the IQ blobs.
            measure(
                "readout"*amp(1.),
                "resonator",
                dual_demod.full("cos", "sin", I0),
                dual_demod.full("minus_sin", "cos", Q0),
            )
            align("qubit", "resonator")
            wait(600*u.ns)
            play("x180"*amp(0.72), "qubit" )
            wait(24*u.ns)
            align("qubit", "resonator")
            # The integration weights have changed to maximize the SNR after having calibrated the IQ blobs.
            measure(
                "readout"*amp(1.),
                "resonator",
                dual_demod.full("cos", "sin", I1),
                dual_demod.full("minus_sin", "cos", Q1),
            )
            # Wait for the qubit to decay to the ground state
            wait(thermalization_time * u.us, "resonator")
            # Save the 'I' & 'Q' quadratures to their respective streams
            save(I0, I0_st)
            save(Q0, Q0_st)
            save(I1, I1_st)
            save(Q1, Q1_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I0_st.save_all("I0")
        Q0_st.save_all("Q0")
        I1_st.save_all("I1")
        Q1_st.save_all("Q1")

# Send the QUA program to the OPX, which compiles and executes it
job = addjob(qmprog, qm)

In [ ]:
# Wait for results
results = fetching_tool(job, data_list=["I0", "Q0","I1", "Q1"])
I0,Q0,I1,Q1=results.fetch_all()

In [ ]:
def rot(I,Q,theta,scale=1e4):
    theta = theta*np.pi/180
    return scale*(I*np.cos(theta)-Q*np.sin(theta)) , scale*(I*np.sin(theta)+Q*np.cos(theta))

In [ ]:
fig,ax = plt.subplots()
I0r, Q0r = rot(I0,Q0,0)
I1r, Q1r = rot(I1,Q1,0)
ax.plot(I0r*1e4, Q0r*1e4, label=r"$|0\rangle$")
ax.plot(I1r*1e4, Q1r*1e4, label=r"$|1\rangle$")
ax.set_ylabel("Q")
ax.set_xlabel("I")
ax.set_aspect("equal")

- Trouver la rotation pour obtenir toute l'information sur l'état du qubit sur $I$
- Estimer l'erreur de lecture
- Quelle est la température du qubit ?

# Trajectoires quantiques

In [ ]:
# Parameters Definition
n_meas = 5000  
###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  
    I = declare(fixed)  
    Q = declare(fixed)  
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_meas, n + 1):  # QUA for_ loop for averaging
        measure(
            "readout",
            "resonator",
            dual_demod.full("cos", "sin", I),
            dual_demod.full("minus_sin", "cos", Q),
        )
        save(I, I_st)
        save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.save_all("I")
        Q_st.save_all("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = addjob(qmprog, qm)

In [ ]:
# Wait for results
results = fetching_tool(job, data_list=["I", "Q"])
I,Q=results.fetch_all()

# Générateur quantique de nombres aléatoires

In [ ]:
# Parameters Definition
n_meas = 5000  
thermalization_time = 80 #in µs

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  
    I = declare(fixed)  
    Q = declare(fixed)  
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_meas, n + 1):  # QUA for_ loop for averaging
        play("pi_half","qubit")
        measure(
            "readout",
            "resonator",
            dual_demod.full("cos", "sin", I),
            dual_demod.full("minus_sin", "cos", Q),
        )
        save(I, I_st)
        save(Q, Q_st)
        wait(thermalization_time*u.us)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.save_all("I")
        Q_st.save_all("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = addjob(qmprog, qm)

In [ ]:
# Wait for results
results = fetching_tool(job, data_list=["I", "Q"])
I,Q=results.fetch_all()

In [ ]:
theta = -292*np.pi/180
Ir =  I*np.cos(theta)-Q*np.sin(theta)
Ir *= 1e3
fig,ax=plt.subplots(figsize=(12,4))
ax.plot(Ir)
ax.plot(Ir>.8)

- Extraire une séquence de nombres aléatoires à partir des données
- Est-elle biaisée ? Pourquoi ?

# Tomographie quantique

- Le qubit semble être la moitié du temps dans $|0\rangle$ et l’autre moitié du temps dans $|1\rangle$. Mais il devrait être dans un état quantique déterministe bien défini. Comment peut-on le prouver ?
- En ajoutant des impulsions avant la mesure, essayer d’estimer l’état quantique du qubit.